In [ ]:
#general definitions and imports
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import logging
logging.Logger.disabled=True

import os
import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging( logging_level='CRITICAL')

import numpy as np
import matplotlib.pyplot as plt

import ipytest
ipytest.autoconfig()
from pytest import approx

from pathlib import Path

from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice.Netlist import SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Spice.Netlist import Circuit
from PySpice.Probe.Plot import plot
from PySpice.Math import *

import schemdraw
import schemdraw.elements as elm
from schemdraw import dsp
from schemdraw import logic

from IPython.display import display, HTML

## setup spice library path
directory_path = Path(os.path.abspath('')).resolve().parent.parent
spice_libraries_path = directory_path.joinpath("lib", "spice")
spice_library = SpiceLibrary(spice_libraries_path)
## set the project directory as directory_path
directory_path = Path(os.path.abspath('')).resolve()

In [ ]:
# definitions
        
class VoltageDivider(SubCircuitFactory):
    __name__ = 'voltage_divider'
    __nodes__ = ('n1', 'n2', 'n3' )
    def __init__(self, R=100@u_kΩ, w=0.4, name='voltage_divider'):
        self.__name__ = name
        super().__init__()
        self.R(1, 'n1', 'n2', R * w)
        self.R(2, 'n2', 'n3', R * (1.0-w) )

# construction

for the final circuit input and output buffering and biasing is needed. the buffering is done with opamps. 


In [ ]:
d = schemdraw.Drawing(unit=2.5, inches_per_unit=0.5, lw=1.1)

#first muliplication
JI1 = d.add(elm.AudioJack(xy=[0, d.unit], lftlabel='$IN_1$'))
d.add(elm.Ground("right", xy=JI1.sleeve))
d.add(elm.Line( "right", l=d.unit, at=JI1.tip))
d.add(elm.Line('down').length(d.unit/2))
M1 = d.add(dsp.Circle().label("x"))
d.add(elm.Line( "right", l=d.unit).at(M1.N))
JO1 = d.add(elm.AudioJack("left", switch=True, flip=True, anchor='tip', rgtlabel='$OUT_1$'))
d.add(elm.Ground("right", xy=JO1.sleeve))

d.add(elm.Line("down").at(M1.E).length(d.unit/2))
d.add(elm.Line("left"))
JI2 = d.add(elm.AudioJack(lftlabel='$IN_2$').left().reverse().flip().anchor('tip'))
d.add(elm.Ground("right", xy=JI2.sleeve))

#second multiplication
JI3 = d.add(elm.AudioJack(xy=[-2, -1.5*d.unit], switch=True, lftlabel='$IN_3$').reverse().flip())
d.add(elm.Ground("right", xy=JI3.sleeve))
d.add(elm.Line( "right", l=d.unit, at=JI3.tip))
d.add(elm.Line('down').length(d.unit/2))
M2 = d.add(dsp.Circle().label("x"))
d.add(elm.Line( "right", l=d.unit).at(M2.N))
JO2 = d.add(elm.AudioJack("left", switch=True, flip=True, anchor='tip', rgtlabel='$OUT_2$'))
d.add(elm.Ground("right", xy=JO2.sleeve))

d.add(elm.Line("down").at(M2.E).length(d.unit/2))
d.add(elm.Line("left"))
JI4 = d.add(elm.AudioJack(lftlabel='$IN_4$').left().reverse().flip().anchor('tip'))
d.add(elm.Ground("right", xy=JI4.sleeve))

#connect the two stages
d.add(elm.Line().at(JI3.tipswitch).right().length(d.unit*1.5))
d.add(elm.Line().up().toy(JO1.tipswitch))
d.add(elm.Line().right().tox(JO1.tipswitch))

d.draw()


first we design the input stage. the input voltage for the X signal should be 10mV and 20mV for the control voltage.


In [ ]:
#load the kicad schema
directory_path = Path(os.path.abspath('')).resolve()
kicad_netlist_path = directory_path.joinpath('main', 'main.cir')
parser = SpiceParser(path=str(kicad_netlist_path))


In [ ]:
#ringmodulation
ringmod_schema = parser.build_circuit(ground=5)
ringmod_schema.include(spice_library['TL072c'])
ringmod_schema.include(spice_library['AD633'])

ringmod_schema.V('1', '+15V', ringmod_schema.gnd, 'DC 15')
ringmod_schema.V('2', '-15V', ringmod_schema.gnd, 'DC -15')
ringmod_schema.V('3', 'IN_Xa', ringmod_schema.gnd, 'DC 5V AC 5V SIN(0V 5V 1k)')
ringmod_schema.V('4', 'IN_Ya', ringmod_schema.gnd, 'DC 5V AC 5V SIN(0V 5V 100)')
ringmod_schema.V('5', 'IN_Za', ringmod_schema.gnd, 'DC 0V AC 0V')

#simulator = ringmod_schema.simulator(temperature=25, nominal_temperature=25)
#analysis_ringmod = simulator.transient(step_time=10@u_us, end_time=20@u_ms)

In [ ]:
#amplitude modulation
am_schema = parser.build_circuit(ground=5)
am_schema.include(spice_library['TL072c'])
am_schema.include(spice_library['AD633'])

am_schema.V('1', '+15V', am_schema.gnd, 'DC 15')
am_schema.V('2', '-15V', am_schema.gnd, 'DC -15')
am_schema.V('3', 'IN_Xa', am_schema.gnd, 'DC 10V AC 5V SIN(0V 5V 1k)')
am_schema.V('4', 'IN_Ya', am_schema.gnd, 'DC 5V AC 5V SIN(0V 5V 100)')
am_schema.V('5', 'IN_Za', am_schema.gnd, 'DC 5V AC 5V SIN(0V 2.5V 1k)')

#simulator = am_schema.simulator(temperature=25, nominal_temperature=25)
#analysis_am = simulator.transient(step_time=10@u_us, end_time=20@u_ms)

In [ ]:
#vca
vca_schema = parser.build_circuit(ground=5)
vca_schema.include(spice_library['TL072c'])
vca_schema.include(spice_library['AD633'])

vca_schema.V('1', '+15V', vca_schema.gnd, 'DC 15')
vca_schema.V('2', '-15V', vca_schema.gnd, 'DC -15')
vca_schema.V('3', 'IN_Xa', vca_schema.gnd, 'DC 10V AC 5V SIN(0V 5V 1k)')
vca_schema.V('4', 'IN_Ya', vca_schema.gnd, 'DC 5V AC 5V PULSE(0V 10V 1m 1m 15m 1m)')
vca_schema.V('5', 'IN_Za', vca_schema.gnd, 'DC 0V AC 0V')

#simulator = vca_schema.simulator(temperature=25, nominal_temperature=25)
#analysis_vca = simulator.transient(step_time=10@u_us, end_time=20@u_ms)

In [ ]:
display(HTML('''
{{< bom produkt main mount>}}
{{< callout produkt main mount panel>}}
{{< report produkt main mount>}}
'''))

# calibration

there is no calibration needed. but the potentiomenter knobs have to be aliged to center position.

# usage

the input jacks are wired to 5 volts when nothing is connected.

all channels are mixed to the out jack. when something is connected to the channel out, this channel is removed from the overall mix.

_mixer_
* connect the different channels from audio or cv sources to the in jacks.
* connect the out jack to something
* turn the pots clockwise to adjust the volume.

_attenuverter_
* connect all or a single channel. 
* when you turn the pot knob counter clockwise the signal is inverted.
